In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

# 1. Cargar el Master original
# ---------------------------------------------------------
project_root = os.path.dirname(os.getcwd())
# Ajusta esta ruta si es necesario según tu estructura de carpetas
try:
    df = pd.read_parquet(os.path.join(project_root, "data_cleaned", "master.parquet"))
except:
    # Ruta alternativa si estás en la misma carpeta
    df = pd.read_parquet("data_cleaned/master.parquet")

print(f"Dataset original cargado: {df.shape}")

# 2. Generar Cluster de CLIENTES (RFM - El código que acabamos de validar)
# ---------------------------------------------------------
print("Generando Clusters de Clientes...")
fecha_ref = df['Fecha'].max() + pd.Timedelta(days=1)
clientes_rfm = df.groupby("ID_Cliente").agg({
    "Fecha": lambda x: (fecha_ref - x.max()).days,
    "ID_Venta": "nunique",
    "ingreso": "sum",
    "Cantidad": "sum"
}).reset_index()
clientes_rfm.columns = ["ID_Cliente", "Recencia", "Frecuencia", "Monto", "Unidades"]

# Estandarizar y Clusterizar (k=3)
scaler_cli = StandardScaler()
X_cli = scaler_cli.fit_transform(clientes_rfm[["Recencia", "Frecuencia", "Monto", "Unidades"]])
kmeans_cli = KMeans(n_clusters=3, random_state=42, n_init="auto")
clientes_rfm["Cluster_Cliente"] = kmeans_cli.fit_predict(X_cli)

# 3. Generar Cluster de PRODUCTOS (Basado en la lógica de tu PDF anterior)
# ---------------------------------------------------------
print("Generando Clusters de Productos...")
# Agrupar por producto y mes [cite: 2697-2704]
df['year'] = df['Fecha'].dt.year
df['month'] = df['Fecha'].dt.month
prod_mes = df.groupby(["ID_Producto", "year", "month"], as_index=False)["ingreso"].sum()

# Pivotear para tener meses como columnas [cite: 2711-2717]
mat_prod = prod_mes.pivot_table(index="ID_Producto", columns="month", values="ingreso", fill_value=0)

# Crear features de comportamiento (Total, Volatilidad, Coef Variación) 
features_prod = pd.DataFrame(index=mat_prod.index)
features_prod["total_year"] = mat_prod.sum(axis=1)
features_prod["std_month"] = mat_prod.std(axis=1)
# Coeficiente de variación (evitando división por cero)
features_prod["coef_var"] = features_prod["std_month"] / (features_prod["total_year"] / 12 + 1e-6)

# Estandarizar y Clusterizar (k=2 según tu análisis previo) [cite: 2957-2960]
scaler_prod = StandardScaler()
X_prod = scaler_prod.fit_transform(features_prod)
kmeans_prod = KMeans(n_clusters=2, random_state=42, n_init="auto")
features_prod["Cluster_Producto"] = kmeans_prod.fit_predict(X_prod)

# Reset index para tener ID_Producto como columna
features_prod = features_prod.reset_index()[["ID_Producto", "Cluster_Producto"]]

# 4. UNIR TODO (Merge)
# ---------------------------------------------------------
print("Uniendo todo en el Master Final...")

# Unir Cluster Cliente
df_final = df.merge(clientes_rfm[["ID_Cliente", "Cluster_Cliente"]], on="ID_Cliente", how="left")

# Unir Cluster Producto
df_final = df_final.merge(features_prod, on="ID_Producto", how="left")

# Limpieza final (eliminar columnas auxiliares si quedaron)
if 'year' in df_final.columns: df_final.drop(columns=['year', 'month'], inplace=True)

# 5. Guardar
# ---------------------------------------------------------
output_path = os.path.join(os.path.dirname(df.head(1).index.name) if df.index.name else ".", "data_cleaned", "master_with_clusters.parquet")
# Si la ruta compleja falla, usa una simple:
if not os.path.exists(os.path.dirname(output_path)):
    output_path = "master_with_clusters.parquet"

df_final.to_parquet(output_path)

print("="*50)
print(f"¡LISTO! Nuevo dataset guardado en: {output_path}")
print(f"Dimensiones finales: {df_final.shape}")
print("="*50)
print("Muestra de las nuevas columnas:")
print(df_final[["ID_Venta", "Cluster_Cliente", "Cluster_Producto"]].head())

Dataset original cargado: (3000, 25)
Generando Clusters de Clientes...
Generando Clusters de Productos...
Uniendo todo en el Master Final...
¡LISTO! Nuevo dataset guardado en: master_with_clusters.parquet
Dimensiones finales: (3000, 27)
Muestra de las nuevas columnas:
   ID_Venta  Cluster_Cliente  Cluster_Producto
0       919                0                 1
1       947                1                 0
2      1317                0                 1
3      1607                2                 0
4      2038                1                 0
